In [1]:
import pickle
# import json
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timezone
# from sklearn.linear_model import Lasso

In [2]:
number_of_entries = 500
link = f"https://api.cryptobay.io/bay/cryptobaygetauctionlist?\
data=%7B%22object%22%3A%7B%7D%2C%22token_type%22%3A1%2C%22state%22%3A%220%22%2C%22\
order_by%22%3A%222%22%2C%22page_num%22%3A0%2C%22page_size%22%3A{number_of_entries}%7D"

try:
    relative_location
except NameError:
    relative_location = ""

zzratio = 1.4       #minium ratio to appear
discount = 0.85     #discount on what the boat should sell at

r = requests.get(link)
raw_json = r.json()

In [3]:
def get_binance_price_coinograph() -> int:
    """
    Example
            {
        "exchange": "binance",
        "pair": "bnbusdt",
        "price": 335.8,
        "ask": 335.8,
        "bid": 335.7
        }
    """
    price_json = requests.get("https://coinograph.io/ticker/?symbol=binance:bnbusdt").json()
    return price_json['price']


In [4]:
def get_binance_price_coingecko() -> float:
    """
    Example
{'id': 'binancecoin',
 'symbol': 'bnb',
 'name': 'Binance Coin',
 'image': 'https://assets.coingecko.com/coins/images/825/large/binance-coin-logo.png?1547034615',
 'current_price': 400.07,
 'market_cap': 61889077351,
 'market_cap_rank': 5,
 'fully_diluted_valuation': 68296906490,
 'total_volume': 1565273152,
 'high_24h': 401.01,
 'low_24h': 372.01,
 'price_change_24h': 25.8,
 'price_change_percentage_24h': 6.89444,
 'market_cap_change_24h': 3991379311,
 'market_cap_change_percentage_24h': 6.89385,
 'circulating_supply': 154533651.9,
 'total_supply': 170533651.9,
 'max_supply': 170533651.9,
 'ath': 686.31,
 'ath_change_percentage': -41.64571,
 'ath_date': '2021-05-10T07:24:17.097Z',
 'atl': 0.0398177,
 'atl_change_percentage': 1005707.34064,
 'atl_date': '2017-10-19T00:00:00.000Z',
 'roi': None,
 'last_updated': '2021-10-01T08:23:03.665Z'}
    """
    url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=binancecoin"
    price_json = requests.get(url).json()[0]['current_price']
    return price_json

In [5]:
def get_ship_stats_selling(bnb_price: float):
    ship_id_list = pd.DataFrame([element['token_id'] for element in raw_json['data']['auctions']])
    tranzaction_id_list = pd.DataFrame([element['idx'] for element in raw_json['data']['auctions']])
    ship_class_list = pd.DataFrame([element['class'] for element in raw_json['data']['auctions']])
    ship_price_list_BNB = pd.DataFrame([ round(int(element['buyout_price']) / 1000000000, 3) for element in raw_json['data']['auctions'] ])
    ship_price_list_USD = pd.DataFrame([ round(int(element['buyout_price']) / 1000000000 * bnb_price, 2) for element in raw_json['data']['auctions'] ])

    boat_space_list = pd.DataFrame([ element['raw_space'] for element in raw_json['data']['auctions']])
    boat_speed_list = pd.DataFrame([ element['raw_speed'] for element in raw_json['data']['auctions']])
    boat_skill_list = pd.DataFrame([ element['raw_skill'] for element in raw_json['data']['auctions']])
    boat_defence_list = pd.DataFrame([ element['raw_defence'] for element in raw_json['data']['auctions']])
    boat_attack_list = pd.DataFrame([ element['raw_attack'] for element in raw_json['data']['auctions']])
    boat_morale_list = pd.DataFrame([ element['raw_morale'] for element in raw_json['data']['auctions']])
    
    concat_DF = pd.concat([ship_id_list, tranzaction_id_list, ship_class_list, \
                        boat_space_list, boat_speed_list, boat_skill_list, \
                        boat_defence_list, boat_attack_list, boat_morale_list,\
                        ship_price_list_USD, ship_price_list_BNB], axis=1)
    
    concat_DF.columns
    
    return concat_DF

In [6]:
# IMPORT MODELS
model_USD = pickle.load(open(f"{relative_location}db/model_USD.sav", 'rb'))
model_BNB = pickle.load(open(f"{relative_location}db/model_BNB.sav", 'rb'))

# zztest = np.array([5,13,15,6,11,5])
# zztest = zztest.reshape(1, -1)

# model_BNB.predict(zztest)


In [7]:
columns = ['transaction_id', 'ship_id', 'ship_class', 'ship_durability', 'ship_owner', \
        'ship_attr_space', 'ship_attr_speed', 'ship_attr_skill', 'ship_attr_defence', 'ship_attr_attack', 'ship_attr_morale', \
        'ship_parts_keel', 'ship_parts_sail', 'ship_parts_side', 'ship_parts_bow', 'ship_parts_cabin', 'ship_parts_stern', \
        'ship_sold_price']

In [8]:
#ship_price_list = [ round(int(element['buyout_price']) / 1000000000 * bnb_price, 2) for element in raw_json['data']['auctions']]

In [9]:
type(get_binance_price_coingecko())

float

In [10]:
try:
    bnb_price
except NameError:
    bnb_price = get_binance_price_coingecko()

# Force refresh for bnb_price 
#bnb_price = get_binance_price_coingecko()

stats_df = get_ship_stats_selling(bnb_price)
stats_df.columns = ['ship_id', 'transaction_id', 'ship_class','ship_attr_space', \
    'ship_attr_speed', 'ship_attr_skill', 'ship_attr_defence', \
    'ship_attr_attack', 'ship_attr_morale', 'selling_price_USD', 'selling_price_BNB']
#stats_df.head(2)

In [11]:
pred_columns = ['ship_attr_space', 'ship_attr_speed', 'ship_attr_skill', \
                'ship_attr_defence', 'ship_attr_attack', 'ship_attr_morale']

X = stats_df[pred_columns]

#X.head()

In [12]:
y_pred_USD = model_USD.predict(X)
y_pred_USD = pd.Series(y_pred_USD)
y_pred_USD = y_pred_USD.round(2)

In [13]:
y_pred_BNB = model_BNB.predict(X)
y_pred_BNB = pd.Series(y_pred_BNB)
y_pred_BNB = y_pred_BNB.round(3)

In [14]:
_df = pd.concat([stats_df, y_pred_USD, y_pred_BNB], axis=1)
_df.rename(columns={0: 'predicted_price_USD', 1: 'predicted_price_BNB'}, inplace=True)
_df['flip_ratio'] = _df['predicted_price_BNB'] / _df['selling_price_BNB']
_df['flip_ratio'] = _df['flip_ratio'].round(2)
gold_df = _df[['ship_id', 'transaction_id', 'selling_price_USD', 'predicted_price_USD', 'selling_price_BNB', 'predicted_price_BNB', 'flip_ratio']]

In [15]:
# gold_df.head()

In [16]:
final_df = (gold_df[gold_df['flip_ratio'] >= zzratio]\
    .sort_values('flip_ratio', ascending=False))

In [17]:
final_df.to_csv(f"{relative_location}db/out/predicted_{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M')}.csv", header=False, index=False)

In [18]:
final_df

,ship_id,transaction_id,selling_price_USD,predicted_price_USD,selling_price_BNB,predicted_price_BNB,flip_ratio
46,14775,24379,49.69,63.26,0.10,0.179,1.79
13,18426,24428,49.69,69.52,0.10,0.172,1.72
36,39680,24396,49.69,60.83,0.10,0.160,1.60
53,17443,24366,49.69,56.43,0.10,0.159,1.59
45,9687,24380,59.63,72.91,0.12,0.190,1.58
35,39382,24398,49.69,64.78,0.10,0.156,1.56
109,33897,24273,49.69,51.61,0.10,0.147,1.47


In [19]:
#final_df['ship_id'].iloc[0]

In [20]:
for i in range(len(final_df)):
    profit = round((final_df['predicted_price_BNB'].iloc[i] * 0.95 * discount - final_df['selling_price_BNB'].iloc[i] - 0.002) * bnb_price, 2)
    print(f"https://marketplace.cryptobay.io/ship/{final_df['ship_id'].iloc[i]}/{final_df['transaction_id'].iloc[i]} \
        ---> actual price: BNB {final_df['selling_price_BNB'].iloc[i]}; ratio {final_df['flip_ratio'].iloc[i]}; \n\
                                                                predicted price: BNB {final_df['predicted_price_BNB'].iloc[i]} \n\
                                                                suggested price: BNB {round(final_df['predicted_price_BNB'].iloc[i] * discount,2)} \n\
                                                                possible profit: ${profit}"
        ) 

https://marketplace.cryptobay.io/ship/14775/24379         ---> actual price: BNB 0.1; ratio 1.79; 
                                                                predicted price: BNB 0.179 
                                                                suggested price: BNB 0.15 
                                                                possible profit: $21.14
https://marketplace.cryptobay.io/ship/18426/24428         ---> actual price: BNB 0.1; ratio 1.72; 
                                                                predicted price: BNB 0.172 
                                                                suggested price: BNB 0.15 
                                                                possible profit: $18.33
https://marketplace.cryptobay.io/ship/39680/24396         ---> actual price: BNB 0.1; ratio 1.6; 
                                                                predicted price: BNB 0.16 
                                                                suggest

In [21]:
# cols = [ col for col in _df.columns if 'attr' in col]
# _df[cols].astype('float').sum(axis=1).sort_values(ascending=False)


In [22]:
#_df.iloc[426]
#_df.iloc[269].values[3:9]

In [23]:

# import numpy as np
# zztest = np.array([6,12,10,11,14,11])

# zztest = np.array(zztest).reshape(1, -1)
# pr_check = model_BNB.predict(zztest)*0.9 
# pr_check

In [24]:
# (pr_check * 0.95 - 0.144) * bnb_price - 0.8